In [26]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
%autoreload
import sys
import copy
sys.path.append('../../style_generation_pipeline/')

import generate_explanations

In [28]:
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import explanation_interfaces
import glob

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Load data

In [29]:
def generate_hiatus_explanations(path_to_input, interp_space_path, interp_space_rep_path, style_feat_clm, style_feat_summary_clm):
    candidates_file = list(glob.glob(path_to_input + '/data/*_candidates.jsonl'))[0]
    queries_file    = list(glob.glob(path_to_input + '/data/*_queries.jsonl'))[0]
    grount_truth_file = list(glob.glob(path_to_input + '/groundtruth/*_groundtruth.npy'))[0]
    q_labels_file = glob.glob(path_to_input + '/groundtruth/*_query-labels.txt')[0]
    c_labels_file = glob.glob(path_to_input + '/groundtruth/*_candidate-labels.txt')[0]
    
    candidates_df = pd.read_json(candidates_file, lines=True)
    queries_df = pd.read_json(queries_file, lines=True)
    
    queries_df['authorID'] = queries_df.authorIDs.apply(lambda x: x[0])
    candidates_df['authorID'] = candidates_df.authorSetIDs.apply(lambda x: x[0])
    
    queries_df = queries_df.groupby('authorID').agg({'fullText': lambda x: list(x)}).reset_index()
    candidates_df = candidates_df.groupby('authorID').agg({'fullText': lambda x: list(x)}).reset_index()
        
    ground_truth_assignment = np.load(open(grount_truth_file, 'rb'))
    candidate_authors = [a[2:-3] for a in  open(c_labels_file).read().split('\n')][:-1]
    query_authors = [a[2:-3] for a in  open(q_labels_file).read().split('\n')][:-1]

    #print(ground_truth_assignment)
    #print(candidate_authors)
    #print(query_authors)
    
    output_json = []
    for q_author_id in query_authors:
    
        q_author_documents = queries_df[queries_df.authorID == q_author_id]['fullText'].tolist()[0]
        c_author_documents = candidates_df.fullText.tolist()

        latent_rank, interp_rank, query_style_reps, query_style_reps_summ, candidates_sim_to_query_exp = generate_explanations.explain_model_prediction_over_author(model_path, interp_space_path, interp_space_rep_path, q_author_documents, c_author_documents, top_c=top_c, style_feat_clm=style_feat_clm, style_feat_summary_clm=style_feat_summary_clm)
        explanation_tmp = copy.copy(explanation_interf)

        candidates_sim_to_query_exp = np.array(candidates_sim_to_query_exp)
        ground_truth_assignment
        instance_json = {
            "Q_fullText": "\n\n".join(["Document {}: \n{}".format(i,d) for i, d in enumerate(q_author_documents)]),
            "Q_authorID": q_author_id,
            "latent_rank" : latent_rank.tolist(),
            "system_rank" : [{"id": x, "title": 'Author {}'.format(x+1), "body": ""} for x in latent_rank.tolist()],
            "interp_rank" : interp_rank.tolist(),
            "explanation" : explanation_tmp
        }
        
    
        for i, s in enumerate(query_style_reps_summ):
            explanation_tmp = explanation_tmp.replace('[style-{}]'.format(i+1), s)
    
        candidate_author_order = []
        for order, c_author_id in enumerate(candidate_authors):
            candidate_sim_to_styles = candidates_sim_to_query_exp[order]
    
            candidate_sim_to_styles = candidates_sim_to_query_exp[order]
            candidate_sim_to_styles_as_color_str = ['rgb(0, {}, 0)'.format(100 * s) for s in candidate_sim_to_styles]
            #print(candidate_sim_to_styles_as_color_str)
            for s_id, color in enumerate(candidate_sim_to_styles_as_color_str):
                explanation_tmp = explanation_tmp.replace('[cand-{}-style-{}]'.format(order+1, s_id+1), color)    
    
            instance_json['a{}_fullText'.format(order)] = "\n\n".join(["Document {}: \n{}".format(i,d) for i, d in enumerate(c_author_documents[order])])
            instance_json['a{}_authorID'.format(order)] = c_author_id
            order+=1
            candidate_author_order.append(candidate_authors.index(c_author_id))
    
        
        #print(explanation_tmp)
        instance_json['explanation'] = explanation_tmp
    
        #Find the ground-truth labels (who wrote the query document)
        query_author_idx = query_authors.index(q_author_id)
        candidate_atuthor_labels = [ground_truth_assignment[query_author_idx][a_idx] for a_idx in candidate_author_order]
        gt_idx = candidate_atuthor_labels.index(1)
        instance_json["gt_idx"] = gt_idx
        output_json.append(instance_json)

    return output_json[0] # it is always one query author per instance

In [8]:
#queries.to_json('/mnt/swordfish-pool2/milad/hiatus-data/test-samples-for-explanations/hrs_06-27-24_english_explainability-example_TA2_input_queries.json')
#candidates.to_json('/mnt/swordfish-pool2/milad/hiatus-data/test-samples-for-explanations/hrs_06-27-24_english_explainability-example_TA2_input_candidates.json')

## Generate documents style explanations:

In [30]:
model_path = 'aa_model-luar'
explanation_interf = explanation_interfaces.exp_interface_1
input_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/'
top_c=3

In [31]:
interp_space_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability_all_data/interpretable_space.pkl'
interp_space_rep_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability_all_data/interpretable_space_representations.json'

output = []
idx=0
for data_point in glob.glob(input_path + '*/'):
    print(data_point)

    
    for feat_clm in ['llm_tfidf_rep', 'llm_con_rep', 'g2v_con_rep', 'g2v_tfidf_rep']:
        instance_exp = generate_hiatus_explanations(data_point, interp_space_path, interp_space_rep_path, feat_clm, None)
        instance_exp['source'] = feat_clm
        output.append(instance_exp)

    open(data_point + '/explanation.html', 'w').write(instance_exp['explanation'])
    idx+=1
    
json.dump(output, open('/mnt/swordfish-pool2/milad/hiatus-data/explainability_all_data/explanations.json', 'w'))

/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples_29_query_80_first/
# clusters: 43
# clusters: 43
# clusters: 43
# clusters: 43
/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples_9_query_20_first/
# clusters: 43
# clusters: 43
# clusters: 43
# clusters: 43
/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples_26_query_79_first/
# clusters: 43
# clusters: 43
# clusters: 43
# clusters: 43
/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples_22_query_70_last/
# clusters: 43
# clusters: 43
# clusters: 43
# clusters: 43
/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples_4_query_14_first/
# clusters: 43
# clusters: 43
# clusters: 43
# clusters: 43
/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples_3_query_6_first/
# clusters: 43
# clusters: 43
# clusters: 43
# clusters: 43
/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples

## Analyze documents style description

In [4]:
interp_space_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/interpretable_space.pkl'
model_path = 'aa_model-luar'
explanation_interf = explanation_interfaces.exp_interface_1
input_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/'
top_c=3

In [5]:
path_to_input = '/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/samples_26_query_79_first/'

In [6]:
candidates_file = list(glob.glob(path_to_input + '/data/*_candidates.jsonl'))[0]
queries_file    = list(glob.glob(path_to_input + '/data/*_queries.jsonl'))[0]
grount_truth_file = list(glob.glob(path_to_input + '/groundtruth/*_groundtruth.npy'))[0]
q_labels_file = glob.glob(path_to_input + '/groundtruth/*_query-labels.txt')[0]
c_labels_file = glob.glob(path_to_input + '/groundtruth/*_candidate-labels.txt')[0]

candidates_df = pd.read_json(candidates_file, lines=True)
queries_df = pd.read_json(queries_file, lines=True)

queries_df['authorID'] = queries_df.authorIDs.apply(lambda x: x[0])
candidates_df['authorID'] = candidates_df.authorSetIDs.apply(lambda x: x[0])

queries_df = queries_df.groupby('authorID').agg({'fullText': lambda x: list(x)}).reset_index()
candidates_df = candidates_df.groupby('authorID').agg({'fullText': lambda x: list(x)}).reset_index()
    
ground_truth_assignment = np.load(open(grount_truth_file, 'rb'))
candidate_authors = [a[2:-3] for a in  open(c_labels_file).read().split('\n')][:-1]
query_authors = [a[2:-3] for a in  open(q_labels_file).read().split('\n')][:-1]

#print(ground_truth_assignment)
#print(candidate_authors)
#print(query_authors)

In [7]:
query_authors

['9265b1f9-31f5-58e5-bfb7-c5979de68d47']

In [8]:
candidate_authors

['07a9a4f0-faed-9b92-fd68-8f47cdc969d5',
 'a83b5d4d-0e79-7ee6-f61b-29b0b059f220',
 'd53e9f35-4c07-3b87-43fc-d52e5f127c6c']

In [9]:
candidate_authors[-1]

'd53e9f35-4c07-3b87-43fc-d52e5f127c6c'

In [10]:
ground_truth_assignment

array([[1, 0, 0]])

In [11]:
matched_candidate_docs = candidates_df[candidates_df.authorID == '07a9a4f0-faed-9b92-fd68-8f47cdc969d5'].fullText.tolist()[0]
other_candidate_docs = candidates_df[candidates_df.authorID != '07a9a4f0-faed-9b92-fd68-8f47cdc969d5'].fullText.tolist()[0] #just pick one other candidate author
query_documents = queries_df.fullText.tolist()[0]

In [12]:
matched_candidate_docs

['The elves had a very distinguished and strict military system, and it included a complex training system that ensured the continued supply of experienced fighters to the army, and this system was based on the soldiers who survived the battles training the new soldiers, as the military system stipulated that the fighters who spent more than 30 years in the war should go to their homeland to train new soldiers in the use of weapons, where the soldier had to prove his competence and strength in order to have the honor of training soldiers.\n\n  But because of the many wars caused by <PERSON> in the first age, and the wars caused by <PERSON> in the second and third ages, where the battles lasted for months, and the siege of castles lasted for years, the elves suffered great losses in their lives, but these losses It was concentrated in the ranks of the swords and spearmen who were always forced to physically interact with the enemy forces, which exposed them to great and permanent danger

In [13]:
other_candidate_docs

["Burning wood has been a local source of warmth for ages, however, while it may provide warmth, it is important to know that burning wood can produce harmful gases such as benzene, and other gases such as carbon monoxide and nitrogen oxides. When organic matter burns, large polycyclic aromatic compounds are released due to combustion. This occurs not only with wood, but also with any other organic matter. Understanding why benzene form in a wood fire, will require us to know what wood is, and how it burns.\n\n  Wood is made up of biomass (matter composed primarily of carbon, hydrogen, and oxygen). This includes things like dry grass, leaves, fibers, animal carcasses, sugar, fat, and coal etc. All of these things contain cellulose or similar molecules, which consists of long chains of aromatic rings.\n\n  When wood is heated up, the molecules composing it start to break apart, in the presence of oxygen, the wood will start to oxidize, this process is what we know as combustion. If ther

In [14]:
query_documents

['The Sahara Desert extends over an area of 9,200,000 square kilometers and is the largest desert in the world. It is one of the driest regions in the world with an average of 25 mm of rain per year. In the past, it was a green area full of rivers, 3 million years ago, until climate change, and the occurrence of several major earthquakes led to the formation of new mountains that blocked the moisture of the sea, and changed the course of the rivers in them, all of this led to the cutting off of water resources and turning it into a dry area with high temperatures, unfit for most types of life.\n\n  Climate change is expected to continue to cause bad consequences for the Sahara desert, as studies indicate that rainfall is expected to decrease by 20 to 30 percent in the next 40 years, and this will lead to more desertification and the destruction of more green spaces on The borders of deserts in general, and will lead to an increase in the borders of the Sahara desert by no less than 7 p

In [15]:
q_documents_reps , q_documents_clusters = generate_explanations.get_documents_style_descriptions(query_documents, model_path, interp_space_path, top_c=3, top_k=5)

# clusters: 14


In [16]:
c_documents_reps , c_documents_clusters = generate_explanations.get_documents_style_descriptions(matched_candidate_docs, model_path, interp_space_path, top_c=3, top_k=5)

# clusters: 14


In [17]:
o_documents_reps , o_documents_clusters = generate_explanations.get_documents_style_descriptions(other_candidate_docs, model_path, interp_space_path, top_c=3, top_k=5)

# clusters: 14


In [18]:
q_documents_clusters

[[(13, 0.6829973459243774), (5, 0.5885012149810791), (3, 0.5295000076293945)],
 [(13, 0.42635929584503174),
  (3, 0.42302098870277405),
  (5, 0.3870851993560791)],
 [(13, 0.7052289247512817), (5, 0.6760463714599609), (12, 0.5887666344642639)],
 [(3, 0.4849829375743866), (13, 0.4269305467605591), (5, 0.3840888440608978)]]

In [19]:
c_documents_clusters

[[(5, 0.3803691864013672),
  (13, 0.28969496488571167),
  (7, 0.25092387199401855)]]

In [20]:
o_documents_clusters

[[(3, 0.9635261297225952), (13, 0.8992846608161926), (12, 0.7395482063293457)]]

In [21]:
q_documents_reps

[[(13,
   ('The approach used by the author is systematic, starting with a general statement and then exploring more details.',
    19.92006319968675)),
  (13,
   ('Revised: The author prefers a diverse writing style with structured sentences.',
    13.515608204437266)),
  (13,
   ('The author expresses strong emotions and perspectives through the use of impactful language.',
    13.069321101808846)),
  (13,
   ('The writer employs diverse sentence structures for improved clarity.',
    12.39637662856642)),
  (13,
   ('The author effectively conveys emotions through descriptive language.',
    11.893747772004609)),
  (5,
   ('The writer uses various voices to create different effects.',
    18.470951754889448)),
  (5,
   ('The use of figurative language conveys complex ideas.',
    13.069321101808846)),
  (5,
   ('Passive voice is utilized in various situations.', 11.683026740688955)),
  (5,
   ('Rewritten: The author utilizes different sentence lengths for conveying diverse emotions a

In [22]:
c_documents_reps

[[(5,
   ('The writer uses various voices to create different effects.',
    18.470951754889448)),
  (5,
   ('The use of figurative language conveys complex ideas.',
    13.069321101808846)),
  (5,
   ('Passive voice is utilized in various situations.', 11.683026740688955)),
  (5,
   ('Rewritten: The author utilizes different sentence lengths for conveying diverse emotions and ideas.',
    11.683026740688955)),
  (5,
   ('The author creates a connection with the reader through direct engagement.',
    10.806516878826645)),
  (13,
   ('The approach used by the author is systematic, starting with a general statement and then exploring more details.',
    19.92006319968675)),
  (13,
   ('Revised: The author prefers a diverse writing style with structured sentences.',
    13.515608204437266)),
  (13,
   ('The author expresses strong emotions and perspectives through the use of impactful language.',
    13.069321101808846)),
  (13,
   ('The writer employs diverse sentence structures for imp

In [217]:
o_documents_reps

[[(1, ('limited range', 90.29001541922472)),
  (1, ('declarative sentences', 87.53142177173797)),
  (1, ('simple sentence structures', 84.48013863066261)),
  (1, ('present tense', 82.12878113649731)),
  (1, ('consistent verb tense', 60.12597809047695)),
  (4, ('present tense', 9.662209545470272)),
  (4, ('short sentences', 9.107428308901413)),
  (4, ('simple sentence structures', 8.960014703252094)),
  (4, ('declarative sentences', 8.142457839231438)),
  (4, ('complex verb forms', 6.77546121578783)),
  (10, ('present tense', 7.246657159102704)),
  (10, ('declarative sentences', 6.106843379423578)),
  (10, ('limited range', 5.209039351109118)),
  (10, ('occasional switches', 4.655197679587739)),
  (10, ('simple sentence structures', 3.840006301393755))]]

In [ ]:
interpretable_space = pkl.load(open(interp_space_path, 'rb'))
    
del interpretable_space[-1] #DBSCAN generate a cluster -1 of all outliers. We don't want this cluster
print("# clusters:", len(interpretable_space))
dimension_to_latent = {key: interpretable_space[key][0] for key in interpretable_space}
dimension_to_style  = {key: interpretable_space[key][1] for key in interpretable_space}